# Imports

In [1]:
import json,os,shutil,yaml,torch
import torch.nn as nn
from models import create_model
from utils.util import get_optimizer, get_loss, get_scheduler
from utils.data_container import get_data_loader
from utils.preprocess import preprocessing_for_metric
from utils.train import train_model, test_model

In [2]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(2, 16)
        self.conv2 = GCNConv(16, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [3]:
con = 'evoconv2-config'
# data 
with open(os.path.join('config', f'{con}.yaml')) as f:
    conf = yaml.safe_load(f)
data_category=['taxi']

print(json.dumps(conf, indent=4))

os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
device = torch.device(0)

model_name = conf['model']['name']
optimizer_name = conf['optimizer']['name']
data_set = conf['data']['dataset']
scheduler_name = conf['scheduler']['name']
loss = get_loss(**conf['loss'])
print("aaaa", device)

loss.to(device)


support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                       Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])

save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

shutil.rmtree(save_folder, ignore_errors=True)
os.makedirs(save_folder)
shutil.rmtree(run_folder, ignore_errors=True)
os.makedirs(run_folder)

with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
    yaml.safe_dump(conf, _f)

data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                      model_name=model_name)





{
    "name": "Evoconv2",
    "tag": "train1",
    "device": 0,
    "data": {
        "dataset": "nogrid",
        "batch_size": 32,
        "X_list": [
            12,
            11,
            10,
            9,
            8,
            7,
            6,
            5,
            4,
            3,
            2,
            1
        ],
        "Y_list": [
            0,
            1,
            2,
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11
        ],
        "_len": [
            672,
            672
        ],
        "Normal_Method": "Standard"
    },
    "preprocess": {
        "hidden_size": 20,
        "normalized_category": "randomwalk"
    },
    "train": {
        "epochs": 3,
        "max_grad_norm": 5,
        "early_stop_steps": 10
    },
    "optimizer": {
        "name": "Adam",
        "Adam": {
            "lr": 0.0005,
            "weight_decay": 0.0005,
            "e

In [ ]:
# data.metadata=

In [ ]:
### # model = HeteroGNN(data.metadata(), hidden_channels=64, out_channels=2,
# #                   num_layers=2)
# # dataset=data_loader
# # dataset.num_node_features=2
# model=GCN()

In [ ]:
# dataset.update({"num_node_features": 2})
# print(dataset)
# # print(dataset.num_node_features)

In [ ]:
# def create_model(model_name, loss, conf, data_category, device,  support=None):
# #     if model_name == 'Evonet2':
#     model = GraphConv_(**conf, support=torch.from_numpy(support).float(), device=device)
#     for name, parameters in model.named_parameters():
#         print(name, ':', parameters.size())
#     return model, MetricNNTrainer(model, loss)

In [ ]:
# from typing import Tuple
# from .evonn2 import EvoNN2
# import torch
# from torch import nn, Tensor



# def create_model(model_name, loss, conf, data_category, device,  support=None):
#     if model_name == 'Evonet2':
# #         model = EvoNN2(**conf, support=torch.from_numpy(support).float(), device=device)
#         model=GraphConv_(**conf, support=torch.from_numpy(support).float(), device=device)
#         for name, parameters in model.named_parameters():
#             print(name, ':', parameters.size())
#         return model, MetricNNTrainer(model, loss)




# class Trainer:
#     def __init__(self, model: nn.Module, loss):
#         self.model = model
#         self.loss = loss

#     def train(self, inputs: Tensor, targets: Tensor, phase: str) -> Tuple[Tensor, Tensor]:
#         raise ValueError('Not implemented.')


# class MetricNNTrainer(Trainer):
#     def __init__(self, model, loss):
#         super(MetricNNTrainer, self).__init__(model, loss)
#         self.train_batch_seen: int = 0

#     def train(self, inputs: Tensor, targets: Tensor, phase: str):
#         if phase == 'train':
#             self.train_batch_seen += 1
#         i_targets = targets if phase == 'train' else None
#         outputs, graph = self.model(inputs, i_targets, self.train_batch_seen)
#         loss = self.loss(outputs, targets, graph)
#         # loss = self.loss(outputs, targets)
#         return outputs, loss



In [ ]:
model, trainer = create_model(model_name,
                                  loss,
                                  conf['model'][model_name],
                                  data_category,
                                  device,
                                  support)

optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
if torch.cuda.device_count() > 1:
    print("use ", torch.cuda.device_count(), "GPUS")
    model = nn.DataParallel(model)
else:
    model.to(device)

In [ ]:
train_model(model=model,
                   dataloaders=data_loader,
                   trainer=trainer,
                   optimizer=optimizer,
                   normal=normal,
                   scheduler=scheduler,
                   folder=save_folder,
                   tensorboard_folder=run_folder,
                   device=device,
                   **conf['train'])


In [ ]:
test_model(folder=save_folder,
                  trainer=trainer,
                  model=model,
                  normal=normal,
                  dataloaders=data_loader,
                  device=device)

In [ ]:
def train(conf, data_category):
    print(json.dumps(conf, indent=4))

    os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
    device = torch.device(0)

    model_name = conf['model']['name']
    optimizer_name = conf['optimizer']['name']
    data_set = conf['data']['dataset']
    scheduler_name = conf['scheduler']['name']
    loss = get_loss(**conf['loss'])
    print("aaaa", device)

    loss.to(device)


    support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                           Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])
    model, trainer = create_model(model_name,
                                  loss,
                                  conf['model'][model_name],
                                  data_category,
                                  device,
                                  support)

    optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
    scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
    if torch.cuda.device_count() > 1:
        print("use ", torch.cuda.device_count(), "GPUS")
        model = nn.DataParallel(model)
    else:
        model.to(device)

    save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
    run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

    shutil.rmtree(save_folder, ignore_errors=True)
    os.makedirs(save_folder)
    shutil.rmtree(run_folder, ignore_errors=True)
    os.makedirs(run_folder)

    with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
        yaml.safe_dump(conf, _f)

    data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                          model_name=model_name)


    train_model(model=model,
                       dataloaders=data_loader,
                       trainer=trainer,
                       optimizer=optimizer,
                       normal=normal,
                       scheduler=scheduler,
                       folder=save_folder,
                       tensorboard_folder=run_folder,
                       device=device,
                       **conf['train'])
    test_model(folder=save_folder,
                      trainer=trainer,
                      model=model,
                      normal=normal,
                      dataloaders=data_loader,
                      device=device)


In [ ]:
# print(conf['taxi'])

In [ ]:
con = 'evoconv2-config'
data = ['bike']
with open(os.path.join('config', f'{con}.yaml')) as f:
    conf = yaml.safe_load(f)
# train(conf, data)

In [ ]:
# conf
data_category=['taxi']

os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
device = torch.device(0)

model_name = conf['model']['name']
optimizer_name = conf['optimizer']['name']
data_set = conf['data']['dataset']
scheduler_name = conf['scheduler']['name']
loss = get_loss(**conf['loss'])
print("aaaa", device)

loss.to(device)


support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                       Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])
model, trainer = create_model(model_name,
                              loss,
                              conf['model'][model_name],
                              data_category,
                              device,
                              support)

optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
if torch.cuda.device_count() > 1:
    print("use ", torch.cuda.device_count(), "GPUS")
    model = nn.DataParallel(model)
else:
    model.to(device)

save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

shutil.rmtree(save_folder, ignore_errors=True)
os.makedirs(save_folder)
shutil.rmtree(run_folder, ignore_errors=True)
os.makedirs(run_folder)

with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
    yaml.safe_dump(conf, _f)

data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                      model_name=model_name)






In [ ]:
# print(data_loader)

In [ ]:
# print(data_loader)

In [ ]:
train_model(model=model,
                   dataloaders=data_loader,
                   trainer=trainer,
                   optimizer=optimizer,
                   normal=normal,
                   scheduler=scheduler,
                   folder=save_folder,
                   tensorboard_folder=run_folder,
                   device=device,
                   **conf['train'])

In [ ]:
# import os
# import numpy as np
# import math
# import json
# import torch
# from tensorboardX import SummaryWriter
# from torch import nn, optim
# from tqdm import tqdm
# import copy, time
# from utils.util import save_model, get_number_of_parameters
# from collections import defaultdict
# from utils.util import loss_calculate
# from utils.evaluate import evaluate
# from utils.util import MyEncoder

# def test_model(folder: str,
#                   trainer,
#                   model,
#                   normal,
#                   dataloaders,
#                   device):

#     save_path = os.path.join(folder, 'best_model.pkl')
#     save_dict = torch.load(save_path)
#     # model.load_state_dict(save_dict['model_state_dict'])

#     # model.eval()
#     steps, predictions, running_targets = 0, list(), list()
#     tqdm_loader = tqdm(enumerate(dataloaders['test']))
#     for step, (inputs, targets) in tqdm_loader:
#         running_targets.append(targets.numpy())

#         with torch.no_grad():
#             inputs = inputs.to(device)
#             targets = targets.to(device)
#             outputs, loss = trainer.train(inputs, targets, 'test')
#             predictions.append(outputs.cpu().numpy())

#     running_targets, predictions = np.concatenate(running_targets, axis=0), np.concatenate(predictions, axis=0)
# #     print("xx", predictions)

#     scores = evaluate(running_targets, predictions,normal)

#     print('test results:')
#     print(json.dumps(scores,cls=MyEncoder, indent=4))
#     with open(os.path.join(folder, 'test-scores.json'), 'w+') as f:
#         json.dump(scores, f,cls=MyEncoder, indent=4)
#     if trainer.model.graph0 is not None:
#         np.save(os.path.join(folder, 'graph0'),trainer.model.graph0.detach().cpu().numpy())
#         np.save(os.path.join(folder, 'graph1'),trainer.model.graph1.detach().cpu().numpy())
#         np.save(os.path.join(folder, 'graph2'),trainer.model.graph2.detach().cpu().numpy())

#     np.savez(os.path.join(folder, 'test-results.npz'), predictions=predictions, targets=running_targets)


In [ ]:
test_model(folder=save_folder,
                  trainer=trainer,
                  model=model,
                  normal=normal,
                  dataloaders=data_loader,
                  device=device)

In [ ]:
import pandas as pd
import numpy as np
a=pd.read_pickle('data/adj_data/adj_mx_taxi.pkl')

In [ ]:
adj=np.array(a)


In [ ]:
torch.save(adj, 'adj.pt')

In [ ]:
import numpy as np

In [ ]:
print(dir(data_set))

In [ ]:
xx=data_loader['test']

In [ ]:
print(type(xx))

In [ ]:
for i, batch in enumerate(xx):
    aa=batch

In [ ]:
aa=np.array(aa)

In [ ]:
print(aa[0].shape)

In [ ]:
from tqdm import tqdm


In [ ]:
tqdm_loader = tqdm(enumerate(data_loader['train']))
x_train=[]
y_train=[]
for step, (inputs, targets) in tqdm_loader:
#     print(step)
    x_train.append(inputs) 
    y_train.append(targets)
    
    
    

In [ ]:
tqdm_loader = tqdm(enumerate(data_loader['test']))
x_test=[]
y_test=[]
for step, (inputs, targets) in tqdm_loader:
#     print(step)
    x_test.append(inputs) 
    y_test.append(targets)

In [ ]:
print(x[92].shape)
# print(y)
# print(x[0][0][1], y[0][0][2])

In [ ]:
torch.save(x_train, 'x_train.pt')
torch.save(y_train, 'y_train.pt')
torch.save(x_test, 'x_test.pt')
torch.save(y_test, 'y_test.pt')